In [2]:
from dotenv import load_dotenv
import dspy
import os

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [32]:
lm = dspy.LM("openai/gpt-4.1-mini", temperature=1, api_key=api_key, max_tokens=32000)
dspy.configure(lm=lm)

In [33]:
import dspy
from datasets import load_dataset

def init_dataset():
    train_split = load_dataset("AI-MO/aimo-validation-aime")['train']
    train_split = [
        dspy.Example({
            "problem": x['problem'],
            'solution': x['solution'],
            'answer': x['answer'],
        }).with_inputs("problem")
        for x in train_split
    ]
    import random
    random.Random(0).shuffle(train_split)
    tot_num = len(train_split)

    test_split = load_dataset("MathArena/aime_2025")['train']
    test_split = [
        dspy.Example({
            "problem": x['problem'],
            'answer': x['answer'],
        }).with_inputs("problem")
        for x in test_split
    ]

    train_set = train_split[:int(0.5 * tot_num)]
    val_set = train_split[int(0.5 * tot_num):]
    test_set = test_split * 5

    return train_set, val_set, test_set

In [34]:
train_set, val_set, test_set = init_dataset()

In [35]:
test_point = 0
print("Problem:")
print(train_set[test_point]['problem'])
# print("\n\nSolution:")
# print(train_set[test_point]['solution'])
print("\n\nAnswer:")
print(train_set[test_point]['answer'])

Problem:
In isosceles trapezoid $ABCD$, parallel bases $\overline{AB}$ and $\overline{CD}$ have lengths $500$ and $650$, respectively, and $AD=BC=333$. The angle bisectors of $\angle{A}$ and $\angle{D}$ meet at $P$, and the angle bisectors of $\angle{B}$ and $\angle{C}$ meet at $Q$. Find $PQ$.


Answer:
242


In [36]:
print(f"{len(test_set)=}, {len(train_set)=}, {len(val_set)=}")

len(test_set)=150, len(train_set)=45, len(val_set)=45


In [37]:
class GenerateResponse(dspy.Signature):
    """Solve the problem and provide the answer in the correct format."""
    problem = dspy.InputField()
    answer = dspy.OutputField()

program = dspy.ChainOfThought(GenerateResponse)

In [38]:
def metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    """
    An evaluation metric for our evaluator
    """
    correct_answer = int(example['answer'])
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        return 0
    return int(correct_answer == llm_answer)

In [39]:
import dspy
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=metric,
    num_threads=32,
    display_table=True,
    display_progress=True
)

evaluate(program)

Average Metric: 75.00 / 150 (50.0%): 100%|██████████| 150/150 [00:01<00:00, 139.84it/s]

2025/10/08 12:11:27 INFO dspy.evaluate.evaluate: Average Metric: 75 / 150 (50.0%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,We are given the problem of finding all integer bases \(b > 9\) fo...,70,✔️ [1]
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,We are given $\triangle ABC$ with points on sides $AB$ and $AC$ di...,588,✔️ [1]
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"There are 9 players, each choosing one of three flavors: chocolate...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,"We want to find the number of integer ordered pairs \((x,y)\) with...",117,✔️ [1]
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We are dealing with 8-digit numbers formed from the digits 1 throu...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We have a regular 24-gon with vertices \( S = \{ v_0, v_1, \ldots,...",113,✔️ [1]
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We have an 11-sided polygon \( A_1 A_2 \ldots A_{11} \) with the f...,19,✔️ [1]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,We are given a sequence \((x_k)\) defined as: \[ x_1 = \frac{25}{1...,616,
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,Given a right triangle \(\triangle ABC\) with \(\angle A = 90^\cir...,104,✔️ [1]


EvaluationResult(score=50.0, results=<list of 150 results>)

In [40]:
def metric_with_feedback(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = int(example['answer'])
    written_solution = example.get('solution', '')
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        feedback_text = f"The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}', which couldn't be parsed as a python integer. Please ensure your answer is a valid integer without any additional text or formatting."
        feedback_text += f" The correct answer is '{correct_answer}'."
        if written_solution:
            feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems and ensure your final answer is a valid integer."
        return dspy.Prediction(score=0, feedback=feedback_text)

    score = int(correct_answer == llm_answer)

    feedback_text = ""
    if score == 1:
        feedback_text = f"Your answer is correct. The correct answer is '{correct_answer}'."
    else:
        feedback_text = f"Your answer is incorrect. The correct answer is '{correct_answer}'."
    
    if written_solution:
        feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems."

    return dspy.Prediction(score=score, feedback=feedback_text)

In [41]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    #Changing this auto value to higher values will lead to better perfromance
    auto="light",
    num_threads=32,
    track_stats=True,
    reflection_minibatch_size=3,
    reflection_lm=dspy.LM(model="gpt-5", temperature=1.0, max_tokens=32000, api_key=api_key)
)

optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)

2025/10/08 12:11:36 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 560 metric calls of the program. This amounts to 6.22 full evals on the train+val set.
2025/10/08 12:11:36 INFO dspy.teleprompt.gepa.gepa: Using 45 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/560 [00:00<?, ?rollouts/s]2025/10/08 12:11:36 INFO dspy.evaluate.evaluate: Average Metric: 18.0 / 45 (40.0%)
2025/10/08 12:11:36 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4
GEPA Optimization:   8%|▊         | 45/560 [00:00<00:05, 98.57rollouts/s]2025/10/08 12:11:36 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4


Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:00<00:00, 127.27it/s]

2025/10/08 12:11:36 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/10/08 12:11:36 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: Task: Solve the given math problem and return your work in a strict two-part output format.

Input format:
- You will receive a single field named "problem" containing the full problem statement as plain text.

Output format:
- Provide exactly two top-level sections, in this order:
  1) reasoning
  2) answer
- The "reasoning" section should contain a concise, correct solution outline with key steps and justifications.
- The "answer" section must contain only the final result requested by the problem (e.g., a single integer or expression), with no extra words, symbols, or formatting.

General solution guidelines:
- Aim for a short, logically structured solution. Prefer algebraic identities, modular arithmetic, structural observations, and bounding arguments over brute force. If enumeration is required, keep it mi

2025/10/08 12:11:37 INFO dspy.evaluate.evaluate: Average Metric: 17.0 / 45 (37.8%)
2025/10/08 12:11:37 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Full valset score for new program: 0.37777777777777777
2025/10/08 12:11:37 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Full train_val score for new program: 0.37777777777777777
2025/10/08 12:11:37 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Individual valset scores for new program: [0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2025/10/08 12:11:37 INFO dspy.teleprompt.gepa.gepa: Iteration 1: New valset pareto front scores: [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2025/10/08 12:11:37 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Full valset pareto front score: 0.4888888888888889
2025/10/08 12:11:37 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Updated valset pa

Average Metric: 1.00 / 3 (33.3%): 100%|██████████| 3/3 [00:00<00:00, 136.32it/s]

...

2025/10/08 13:05:38 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)


2025/10/08 13:07:28 INFO dspy.teleprompt.gepa.gepa: Iteration 10: Proposed new text for predict: Task: Solve contest-style math problems and output only the final numeric answer in the exact format requested by the problem. Do not include steps, explanations, or extra text unless the prompt explicitly asks for reasoning.

Output rules:
- Read the question carefully and identify exactly what scalar quantity is requested (e.g., CE, AC^2, m+n, an integer count, a simplified fraction).
- Default output is a single integer unless a simplified rational is explicitly required.
- Output only that value, simplified:
  - Integers in standard form (no commas, no leading zeros unless explicitly required).
  - If asked for m+n given a reduced fraction m/n, reduce the fraction first, then compute m+n and output that integer.
  - If a simplified rational is explicitly required, reduce it fully (no radicals; fractions in lowest terms).
- Do not output radicals, approximations, units, prose, labels, or

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [02:13<00:00, 44.64s/it] 

2025/10/08 13:14:01 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)


2025/10/08 13:15:53 INFO dspy.teleprompt.gepa.gepa: Iteration 11: Proposed new text for predict: Task: Solve contest-style math problems and output only the final numeric answer in the exact format requested by the problem. Do not include steps, explanations, reasoning, or extra text unless the prompt explicitly asks for it.

Input format: A single problem statement in plain text (e.g., AMC/AIME/ARML-style). You must parse the request and compute exactly the scalar quantity asked for.

Output rules:
- Output only the requested scalar value, simplified:
  - Default to a single integer unless the problem explicitly requires a simplified rational or other specific format (e.g., m+n).
  - Integers: no commas, no leading zeros (unless explicitly required).
  - Fractions: fully reduced, in lowest terms.
  - If asked for m+n given a reduced fraction m/n, reduce m/n first, then compute m+n and output that integer.
  - Do not output radicals, approximations, units, prose, labels, punctuation, o

In [42]:
print(optimized_program.predict.signature.instructions)

You are given a single math problem. Solve it correctly and return your work in a strict, parseable format.

Output format:
- Produce exactly two sections in this order:
  1) A section header "### reasoning" followed by your step-by-step solution and checks.
  2) A section header "### answer" followed by ONLY the final numeric answer as a plain integer (no LaTeX, no words, no punctuation, no boxes, no quotes, no units).
- The content under "### answer" must be a valid Python int literal. Do not include any additional text, symbols, or formatting. Example: 242

General guidelines:
- Use exact arithmetic whenever possible; avoid decimal approximations unless explicitly required. If the final result is an integer, do not present approximations.
- If the problem asks for a remainder modulo m, ensure your answer is the unique integer in [0, m−1].
- If the problem asks for m+n where a quantity equals m/n in lowest terms, reduce the fraction exactly and then compute m+n.
- Do not include extr

In [43]:
evaluate(optimized_program)

Average Metric: 50.00 / 150 (33.3%): 100%|██████████| 150/150 [00:01<00:00, 113.18it/s]

2025/10/08 17:38:10 INFO dspy.evaluate.evaluate: Average Metric: 50 / 150 (33.3%)


,problem,example_answer,reasoning,pred_answer,metric
0,Find the sum of all integer bases $b>9$ for which $17_b$ is a divi...,70,We are given a problem involving integer bases b > 9. Define the n...,49,
1,"On $\triangle ABC$ points $A, D, E$, and $B$ lie in that order on ...",588,"We are given triangle \(ABC\) with points \(A, D, E, B\) collinear...",594,
2,The 9 members of a baseball team went to an ice-cream parlor after...,16,"We have 9 players choosing flavors from {chocolate, vanilla, straw...",16,✔️ [1]
3,"Find the number of ordered pairs $(x,y)$, where both $x$ and $y$ a...",117,We are given the equation \[12x^2 - xy - 6y^2 = 0\] and want to fi...,119,
4,There are $8!= 40320$ eight-digit positive integers that use each ...,279,We are asked about eight-digit integers formed from the digits 1 t...,279,✔️ [1]
...,...,...,...,...,...
145,Let $S$ be the set of vertices of a regular $24$-gon. Find the num...,113,"We are given a regular 24-gon with vertex set \(S\), and we want t...",3331,
146,Let $A_1 A_2 A_3 \ldots A_{11}$ be an $11$-sided non-convex simple...,19,We are given an 11-sided polygon \( A_1 A_2 \cdots A_{11} \) with ...,19,✔️ [1]
147,"Let $x_1, x_2, x_3, \ldots$ be a sequence of rational numbers defi...",248,"Given the sequence defined by: \[ x_1 = \frac{25}{11}, \quad x_{k+...",438,
148,Let $\triangle ABC$ be a right triangle with $\angle A = 90^\circ$...,104,"First, summarize the problem data: - Triangle \( ABC \) is right-a...",98,


EvaluationResult(score=33.33, results=<list of 150 results>)